In [1]:
from torch import *
import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

data = pd.read_csv('advertising.csv')
data.head()

X = data.drop(['Timestamp', 'Clicked on Ad', 'Ad Topic Line', 'Country', 'City'], axis=1)
y = data['Clicked on Ad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_columns = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Male']


ct = make_column_transformer(
    (MinMaxScaler(), num_columns),
    (StandardScaler(), num_columns),
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
ones_column = torch.ones(700, 1)
ones_column_test = torch.ones(300, 1)
X_train = torch.cat((X_train, ones_column), dim=1)
X_train = X_train.to(device)
X_test = torch.cat((X_test, ones_column_test), dim=1)
X_test = X_test.to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_train = y_train.unsqueeze(0)
y_train = y_train.to(device)

In [18]:
X_train

tensor([[ 0.7318,  0.4762,  0.7639,  ...,  1.5211,  1.0438,  1.0000],
        [ 0.2285,  0.3095,  0.7865,  ..., -0.7576, -0.9580,  1.0000],
        [ 0.6259,  0.1429,  0.7909,  ...,  0.7343, -0.9580,  1.0000],
        ...,
        [ 0.9990,  0.6190,  0.5791,  ...,  0.0339,  1.0438,  1.0000],
        [ 0.4090,  0.5476,  0.8962,  ..., -0.9514, -0.9580,  1.0000],
        [ 0.9742,  0.5000,  0.6963,  ...,  0.2978, -0.9580,  1.0000]])

In [19]:
feature_length = len(X_train[0])

weights = torch.zeros((feature_length, 1), device=device, requires_grad=True)

In [20]:
sig = torch.nn.Sigmoid()
for epoch in range(0, 20):
  out = sig(torch.matmul(torch.t(weights), X_train .T))
  loss = torch.nn.functional.binary_cross_entropy(out, y_train.float())
  grad = torch.autograd.grad(loss, weights, create_graph=True)[0]
  hessian = torch.zeros(len(weights), len(weights))  # Initialize Hessian matrix
  for i in range(0, 10):
      # Compute second derivative of loss w.r.t. params[i] and params[j]
      grad2 = torch.autograd.grad(grad[i.long()], weights, retain_graph=True)[0]
      hessian[i.long()] = grad2.squeeze()
  hessian += 1e-5 * torch.eye(len(weights))
  hessian = torch.linalg.inv(hessian)
  hessian = hessian.to(device)
  update = torch.matmul(hessian, grad)
  weights = weights - update.detach()
  print('--------------------------------------------------------------------------------------------------')
  print('epoch number: ')
  print(epoch)
  print('loss: ')
  print(loss)
  print('--------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------
epoch number: 
tensor(0.)
loss: 
tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
epoch number: 
tensor(1.)
loss: 
tensor(0.2142, grad_fn=<BinaryCrossEntropyBackward0>)
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
epoch number: 
tensor(2.)
loss: 
tensor(0.1313, grad_fn=<BinaryCrossEntropyBackward0>)
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
epoch number: 
tensor(3.)
loss: 
tensor(0.0989

<ipython-input-20-ae94a539f1bb>:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for epoch in range(0, 20):
<ipython-input-20-ae94a539f1bb>:7: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for i in range(0, 10):


In [21]:
out = sig(torch.matmul(torch.t(weights), X_test.T))
out

tensor([[9.9268e-01, 9.9997e-01, 9.9973e-01, 9.9929e-01, 5.5811e-03, 5.2373e-01,
         9.8450e-03, 9.9996e-01, 4.7517e-03, 1.0000e+00, 1.9650e-03, 9.9975e-01,
         9.9999e-01, 6.4551e-03, 9.9998e-01, 1.0000e+00, 1.0000e+00, 9.9957e-01,
         2.5048e-02, 9.9999e-01, 1.6771e-03, 9.9680e-01, 1.0000e+00, 8.7385e-02,
         1.4586e-03, 1.0000e+00, 1.6268e-02, 1.1782e-03, 9.9999e-01, 9.9967e-01,
         1.3908e-01, 4.5808e-01, 3.5969e-03, 9.9372e-01, 1.0000e+00, 4.8690e-03,
         2.3630e-03, 6.5530e-01, 9.9997e-01, 1.0000e+00, 1.1582e-02, 9.9999e-01,
         6.5877e-01, 1.0000e+00, 5.1881e-03, 4.3709e-02, 9.9076e-01, 1.0000e+00,
         2.3321e-01, 9.9995e-01, 1.0000e+00, 2.6049e-02, 9.9975e-01, 9.1250e-03,
         2.4150e-03, 6.8734e-03, 1.0090e-02, 1.0000e+00, 9.5453e-03, 9.9998e-01,
         3.6078e-03, 9.9972e-01, 1.4036e-02, 1.0000e+00, 9.9884e-01, 9.9992e-01,
         1.0000e+00, 4.6506e-03, 9.9940e-01, 9.9996e-01, 9.9975e-01, 9.9931e-01,
         9.1523e-01, 3.0990e

In [22]:
y_test = torch.tensor(y_test.values, dtype=torch.float32)
y_test = y_test.unsqueeze(0)
y_test = y_test.to(device)
y_test

tensor([[1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
         0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
         0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
         1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
         0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 1.,

In [23]:
count = 0
for i in range(0, 299):
  if(y_test[0][i.long()] == 1 and out[0][i.long()] > 0.5):
    count += 1
  if(y_test[0][i.long()] == 0 and out[0][i.long()] < 0.5):
    count += 1

<ipython-input-23-31868f6fbac8>:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for i in range(0, 299):


In [24]:
print('accuracy on test: ')
print(count/3)

accuracy on test: 
96.33333333333333


In [25]:
out = sig(torch.matmul(torch.t(weights), X_train.T))
out

tensor([[1.7779e-03, 9.9792e-01, 1.1914e-02, 9.9911e-01, 9.9999e-01, 2.8051e-01,
         1.3697e-02, 9.9998e-01, 1.2814e-03, 2.9369e-03, 9.9776e-01, 9.9950e-01,
         9.9998e-01, 6.6899e-03, 1.0000e+00, 2.8136e-02, 2.0651e-02, 2.8909e-03,
         6.2604e-02, 1.6373e-03, 9.9998e-01, 9.9877e-01, 9.9986e-01, 9.9999e-01,
         4.2659e-03, 2.5703e-03, 1.6336e-02, 9.8437e-01, 3.5748e-03, 4.7222e-03,
         5.7390e-03, 7.8989e-02, 9.9933e-01, 2.0737e-03, 6.5272e-03, 1.3439e-01,
         9.6409e-01, 4.2790e-03, 2.0545e-02, 2.6351e-01, 9.9999e-01, 9.9999e-01,
         1.0427e-02, 2.7402e-03, 9.9999e-01, 9.9996e-01, 1.0000e+00, 3.6393e-02,
         9.6450e-01, 7.4064e-01, 5.1211e-01, 4.6492e-03, 7.8254e-02, 9.9829e-01,
         5.0797e-03, 9.9998e-01, 6.6213e-03, 9.9980e-01, 9.9991e-01, 4.5391e-02,
         9.9998e-01, 9.9999e-01, 9.9999e-01, 9.9753e-01, 1.0000e+00, 9.9986e-01,
         9.6354e-01, 1.0000e+00, 7.6270e-03, 9.9999e-01, 1.0413e-02, 2.5461e-03,
         1.4411e-02, 2.0502e

In [26]:
y_train

tensor([[0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
         1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
         0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
         0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
         1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
         1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
         1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
         1., 0., 0., 1., 1.,

In [27]:
count = 0
for i in range(0, 699):
  if(y_train[0][i.long()] == 1 and out[0][i.long()] > 0.5):
    count += 1
  if(y_train[0][i.long()] == 0 and out[0][i.long()] < 0.5):
    count += 1

<ipython-input-27-90bdf584d63d>:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for i in range(0, 699):


In [28]:
print('accuracy on train: ')
print(count/7)

accuracy on train: 
97.42857142857143
